In [ ]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
llm = Ollama(model="mistral",  callbacks=CallbackManager([StreamingStdOutCallbackHandler()]), base_url="http://localhost:11434", num_gpu=1)

In [ ]:
import google.generativeai as palm
import os
import dotenv
dotenv.load_dotenv()
from langchain.llms import GooglePalm

google_api_key=os.environ.get('PALM')
palm.configure(api_key=google_api_key)

prompt = 'Explain the difference between effective and affective with examples'

llm = GooglePalm(google_api_key=google_api_key)


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


modelPath = "BAAI/bge-base-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embedding = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

loader = DirectoryLoader('mytext', glob='*.txt', loader_cls=TextLoader)
documents = loader.load()


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)
import torch
torch.cuda.empty_cache()
vectordb = None
print(torch.cuda.memory_summary(device=None, abbreviated=False))
import gc
torch.cuda.empty_cache()
gc.collect()


In [ ]:
from langchain.vectorstores import Chroma
persist_directory = './db'
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()


In [ ]:
import torch
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
dev

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={'k':5})

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  retriever=retriever, 
                                  return_source_documents=True)
     

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
query = "Who was the main character of the bee movie?"
llm_response = qa_chain(query)
process_llm_response(llm_response)